In [29]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [30]:
#Load pickle file - scaler, encoders

with open('lable_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [31]:
#Load model

model = load_model('model.h5')

In [32]:
# Example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
}

df_test = pd.DataFrame([input_data])

print(df_test)

   CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  


In [33]:
df_test['Gender'] = label_encoder_gender.transform(df_test['Gender'])

In [34]:
encoded = one_hot_encoder_geo.transform(df_test[['Geography']]).toarray()
encoder_df = pd.DataFrame(
    encoded, 
    columns=one_hot_encoder_geo.get_feature_names_out(['Geography']),
    index=df_test.index   # keep row alignment
)

# Concatenate with original
df_test = pd.concat([df_test.drop('Geography', axis=1), encoder_df], axis=1)

In [35]:
df_test

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
df_test_scaled = scaler.transform(df_test)

In [37]:
print(df_test_scaled)

[[-0.52493139  0.91255717  0.09471269 -0.69884427 -0.26134615  0.79931788
   0.64007158  0.97530483 -0.87196872  0.99451504 -0.57658047 -0.57388614]]


In [38]:
prediction = model.predict(df_test_scaled)
print(prediction)

1/1 [==============================] - 0s 101ms/step
[[0.04948663]]


In [39]:
prediction_proba = prediction[0][0]

In [40]:
if prediction_proba > 0.5:
    print('The customer likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
